# 1. Preparando ambiente

## Colocando pasta dos dados como Raiz

In [ ]:
from pyspark.sql import SparkSession

# Cria uma SparkSession
spark = (SparkSession.builder
         .appName("MinhaAppSpark")
         .master("local[*]")  # "local[*]" usa todos os núcleos disponíveis localmente
         .getOrCreate()
         )
spark.sparkContext.setLogLevel("ERROR")


# Verifique se a sessão foi criada
print(spark.version)

# Variáveis

In [ ]:
import sys
import os

# Adiciona o diretório atual ao Python path
sys.path.append(os.path.dirname(os.getcwd()))
   
from libs.storage import Storage

storage_log = Storage("files/sipe-u-log-hist-ate-2025")
storage_pefil = Storage("files/perfil")
storage_data = Storage("data")
storage_servidor = Storage("files/siape-servidor")

limit = 10000
random_seed = 42
storage_log

# Carregar o df selecionado

In [ ]:
df = spark.read.csv(storage_data.get_path(f"dataset.csv"))
df.printSchema()

## Transformação 3

Transformar o log normal em uma matrix incidência.

T = Transação [ch_arvore_senha_log]

A matriz a ser criada é TxT porém com os seguintes detalhes.
Agrupar por [CRITÉRIOS]

CRITÉRIOS:
1. Por órgão: co_orgao_log
2. Por órgão do servidor: co_orgao_servidor
3. Por ano ou mês ou dia: da_atualizacao_log

### Explicação do processo:

Para transformar em uma matrix incidência. Vamos rodar todas as transações que um CPF fez por dia.

Se o log regostrou A em um horário e depois B num horário subsequente no mesmo dia. Vamos crira um aresta, A -> B, com o valor 1, se voltar para A e de A for para B, então a aresta A->B agora vale 2. Se no dia subsequente ele saiu novamente de A -> B, então o valor passa a ser 3.

Lembrando que A -> B é diferente de B -> A. Ou seja, estamos criando um DAG, grafo direcionado.

Serão criados n grafos de acordo com os agrupadores: 'co_orgao_log', 'matricula_inst_legal', 'co_orgao_servidor', 'nu_cpf_usuario_log', 'nu_iden_unica_siape_log'

In [14]:
def fill_graph(graph, origem, destino):
    if origem not in graph:
        graph[origem] = {}

    if destino not in graph[origem]:
        graph[origem][destino] = 1
    else:
        graph[origem][destino] += 1

In [15]:
def generate_graph(df_main):
    index_cpf = 'nu_cpf_usuario_log'
    index_data = 'da_atualizacao_log'
    index_transacao = "TRANSACAO"
    
    tmp_graph = {}

    df_indice_cpf = df_main.groupBy(index_cpf).count().orderBy("count", ascending=False)
    # print("df_indice_cpf", df_indice_cpf.head(5))
    for row_cpf in df_indice_cpf.collect():
        # print(row_cpf)
        cpf = row_cpf[index_cpf]
        # print("cpf", cpf)
        df_tmp_cpf = df_main.filter(df_main[index_cpf] == cpf)
        # print("df_tmp_cpf", df_tmp_cpf.count())

        df_indice_data = df_tmp_cpf.groupBy(index_data).count().orderBy("count", ascending=False)
        # print("df_indice_data", df_indice_data.count())

        # Iterate over unique data of 'cpf','sistema'
        for row_data in df_indice_data.collect():  # Iterate over pandas DataFrame
            # print("data", row_data)
            data = row_data[index_data]
            df_tmp_acao = df_tmp_cpf.filter(df_tmp_cpf[index_data] == data)
            # print("df_tmp_acao", df_tmp_acao.count())

            if df_tmp_acao.count() < 2:
                continue
            count = 1
            # Iterate over unique 'acao' of 'data','cpf','sistema'
            for row_acao in df_tmp_acao.collect():  # Iterate over pandas DataFrame
                # progress.next()
                destino = row_acao[index_transacao]
                if destino in [
                    'SIAPE     SIAPENET  ORGAO               LOGIN', 
                    'SIAPE', 
                    '', 
                    'LOGIN'
                ]:
                    continue
                # Dividir o texto em uma lista de palavras
                # origem = re.findall(pattern, origem)
                if count > 1:
                    # palavras = [destino[i:i+10] for i in range(0, len(destino), 10)]
                    # destino = palavras[-1]
                    fill_graph(tmp_graph, origem, destino)

                origem = destino

                count += 1

    return tmp_graph

In [17]:
def generate_graph_by(df, index, index_name):
    graph = {}
    df_index = df.groupBy(index).count().orderBy("count", ascending=False)
    print(index_name, df_index.count())

    for row_sistema in df_index.collect():
        # print(row_sistema)
        camada = row_sistema[index]
        # print("sistema", sistema)
        df_tmp_main = df.filter(df[index] == camada)
        # print("df_tmp_sistema", df_tmp_sistema.count())

        # Iterate over unique 'cpf' of 'sistema'
        graph[f"{index_name}_{camada}"] = generate_graph(df_tmp_main)

    # loberar memoria do df_tmp_main
    df_tmp_main.unpersist()
    df_index.unpersist()
    return graph

## Gerando o grafo filtrado por Sistema

In [ ]:
# Inicializando o grafo
graph = {}

# Sistema
index = 'SIGLA_SISTEMA'
index_name = 'sistema'
graph.update(generate_graph_by(df, index, index_name))

# Órgão
index = 'co_orgao_log'
index_name = 'orgao'
graph.update(generate_graph_by(df, index, index_name))

## Salvando o grafo em um arquivo

In [20]:
import json
graph_file = storage_pefil.get_path("graph.json")
# Abrindo um arquivo para escrita
with open(graph_file, 'w') as arquivo:
    json.dump(graph, arquivo, indent=4)

## Carregando o grafo

In [ ]:
import json
graph_file = storage_pefil.get_path("graph.json")

with open(graph_file, 'r') as arquivo:
    graph = json.load(arquivo)

print(graph)

## Desenhando o grafo no formato dot e salvando em um arquivo pdf


In [22]:
import graphviz


for camada in graph:
    dot = graphviz.Graph(comment='Autorizações')
    for source in graph[camada]:
        # label = origem[:-20]
        dot.node(source, label=source)
        for target in graph[camada][source]:
            dot.node(target, label=target)
            dot.edge(source, target, label="%d" % graph[camada][source][target])
    dot.render(f'autoriza_{camada}.dot')  # Renderizar e abrir a imagem

# print(dot.source)  # Visualizar o código DOT


In [ ]:
import numpy as np

adjacence_matrix = {}
dicionario = {}
collums = set()
for camada in graph:
    for source in graph[camada]:
        for target in graph[camada][source]:
            collums.add(source)
            collums.add(target)
    if camada not in adjacence_matrix:
        adjacence_matrix[camada] = {}    
    
    adjacence_matrix[camada] = np.zeros((len(collums), len(collums)))
    dicionario[camada] = dict(zip(collums, range(len(collums))))
    # print(dicionario[sistema])

for camada in graph:
    d = dicionario[camada]
    for source in graph[camada]:
        for target in graph[camada][source]:
            adjacence_matrix[camada][d[source]][d[target]] = graph[camada][source][target]
adjacence_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans

for camada in adjacence_matrix:
    print(camada)
    # Gerar dados de exemplo
    X = adjacence_matrix[camada]
    k_values = range(1, int(1.5*math.sqrt(X.shape[1])))
    inercia = []
    for k in k_values:
        print("k", k)
        # Criar o modelo de clusterização K-Means
        kmeans = KMeans(n_clusters=k, random_state=random_seed)
        kmeans.fit(X)
        
        # Prever os clusters
        y_kmeans = kmeans.predict(X)
        print(y_kmeans)
        inercia.append(kmeans.inertia_)
    
    # Plotar a inércia para cada valor de k
    plt.plot(k_values, inercia, marker='o')
    plt.xlabel('Número de Clusters (k)')
    plt.ylabel('Inércia (Soma dos Erros Quadrados)')
    plt.title('Método do Cotovelo para Escolha de k')
    plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# Exemplo de uma matriz de incidência (5 objetos com 4 características)
matriz_incidencia = np.array([
    [1, 0, 0, 1],  # Objeto 1
    [1, 1, 0, 0],  # Objeto 2
    [0, 1, 1, 0],  # Objeto 3
    [1, 0, 0, 1],  # Objeto 4
    [0, 1, 1, 1],  # Objeto 5
])

# Número de clusters (definido pelo usuário)
num_clusters = 5

# Criar o modelo K-Means
kmeans = KMeans(n_clusters=num_clusters)

# Ajustar o modelo à matriz de incidência
kmeans.fit(matriz_incidencia)

# Prever os clusters
rótulos = kmeans.predict(matriz_incidencia)

# Exibir os rótulos dos clusters
print("Rótulos dos clusters:", rótulos)

# Exibir os centróides dos clusters
print("Centróides dos clusters:", kmeans.cluster_centers_)
